In [34]:
import jax
import jax.numpy as jnp
import numpy as np
from jax.experimental.ode import odeint
import matplotlib.pyplot as plt
import pickle
from functools import partial # reduces arguments to function by making some subset implicit

from jax.experimental import stax
from jax.experimental import optimizers

# visualization
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from moviepy.editor import ImageSequenceClip
from functools import partial
import proglog
from PIL import Image

import os
import shutil
import random; random.seed(42)

import pandas as pd
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import torch
from torch import nn
from sklearn.model_selection import train_test_split
#import tensorflow as tf # tensorflow-gpu==2.0.0
#from tensorflow.python.client import device_lib 
#print(device_lib.list_local_devices())
#import cv2

from simulate_data import generate_train_ideal
from simulate_data import generate_train_noisy

In [3]:
x_train, xt_train, y_train, x_test, xt_test, y_test = generate_train_ideal()

generating data...


In [37]:
print(x_train[0])
print(x_test[0])

[1.3463968 2.3561945 0.        0.       ]
[1.3463968 2.3561945 0.        0.       ]


In [4]:
print(x_train)

[[ 1.3463968e+00  2.3561945e+00  0.0000000e+00  0.0000000e+00]
 [-4.2738873e-01 -1.1811658e+00 -4.6661725e+00 -6.3316566e-01]
 [-1.2464720e+00 -4.1255212e-01  4.4142170e+00 -6.3992822e-01]
 ...
 [-1.3062775e-01  2.0027132e+02  6.8459344e+00 -6.4956493e+00]
 [ 8.2663119e-01  2.0279190e+02 -4.7293258e+00  2.3417275e+00]
 [-1.2612591e+00  1.9704057e+02 -2.3600051e+00 -4.6696014e+00]]


In [8]:
x_xt_train = []
for data_x, data_xt in zip(x_train, xt_train):
    x_xt_train.append(np.concatenate((data_x, data_xt), axis=None))

In [38]:
x_xt_train, x_xt_test, y_train, y_test = train_test_split(x_xt_train, y_train, test_size=0.2, random_state=42)

In [39]:
class DoublePendulumDataset(torch.utils.data.Dataset):
    def __init__(self,X_list,y_list):
        self.sample_list = list(zip(X_list, y_list))
    
    def __getitem__(self,index):
        X_sample,y_sample = self.sample_list[index]
        return torch.from_numpy(np.array(X_sample)).float(),torch.from_numpy(np.array(y_sample)).float()
    
    def __len__(self):
        return len(self.sample_list)

In [40]:
BATCH_SIZE = 256

# load in all separate files
trainDataSet = DoublePendulumDataset(x_xt_train,y_train)
trainDataLoader = torch.utils.data.DataLoader(trainDataSet, batch_size=BATCH_SIZE)
testDataSet = DoublePendulumDataset(x_xt_test, y_test)
testDataLoader = torch.utils.data.DataLoader(testDataSet, batch_size=BATCH_SIZE)

In [41]:
print(trainDataSet[0][0].shape)
print(trainDataSet[0][1].shape)
print(testDataSet[0][0].shape)
print(testDataSet[0][1].shape)

torch.Size([8])
torch.Size([4])
torch.Size([8])
torch.Size([4])


In [84]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = input_size

        #Defining the layers
        # LSTM layer
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True)
        
        # Fully connected layer
        self.fc = nn.Linear(32, 4)
    
    def forward(self, x):
        out, _= self.lstm1(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [86]:
# Instantiate the model with hyperparameters
model = LSTMModel(8, 32, 3)
model.train()

# Define hyperparameters
n_epochs = 5
lr=0.01

# Define Loss, Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [87]:
print(model)
for name in model.named_parameters():
     print(name[0])

LSTMModel(
  (lstm1): LSTM(8, 32, num_layers=3, batch_first=True)
  (fc): Linear(in_features=32, out_features=4, bias=True)
)
lstm1.weight_ih_l0
lstm1.weight_hh_l0
lstm1.bias_ih_l0
lstm1.bias_hh_l0
lstm1.weight_ih_l1
lstm1.weight_hh_l1
lstm1.bias_ih_l1
lstm1.bias_hh_l1
lstm1.weight_ih_l2
lstm1.weight_hh_l2
lstm1.bias_ih_l2
lstm1.bias_hh_l2
fc.weight
fc.bias


In [88]:
print('Training Start')
for epoch in range(n_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for x_batch, y_batch in trainDataLoader:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = x_batch, y_batch

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

Training Start


RuntimeError: input must have 3 dimensions, got 2